# Notebook - Création d'un dataframe de requête pour les utilisateurs

In [ ]:
#Librairies 
import itertools
import pandas as pd 
import random
import requests
import numpy as np
import copy
import json 

## Définition des paramètres

In [150]:
# les paramètres du jeu de données : 

request_var = ["language","city","date","mobile"]
city = ["amsterdam","copenhagen","madrid","paris","rome","sofia","valletta","vienna","vilnius"]
date = list(range(0,45))
language = ["austrian", "belgian", "bulgarian", "croatian", "cypriot", "czech", "danish", "dutch", "estonian", "finnish", "french",
"german", "greek", "hungarian", "irish", "italian", "latvian", "lithuanian", "luxembourgish", "maltese", "polish", "portuguese", 
            "romanian", "slovakian", "slovene", "spanish", "swedish"]
mobile = [0,1]

dic_language_country = pd.Series({"amsterdam":"dutch",
                                   "copenhagen":"danish",
                                   "madrid":"spanish",
                                   "paris" : "french",
                                   "rome": "italian",
                                   "sofia": "bulgarian",
                                   "valletta": "maltese",
                                   "vienna" : "austrian",
                                   "vilnius" : "lithuanian"})

dic_similar_language = {"french" : ["belgian"],
                       "belgian" : ["french"], 
                       "german" : ["austrian","luxembourgish"],
                       "austrian" : ["german","luxembourgish"],
                       "luxembourgish" : ["german","austrian"]}

In [200]:
# les paramètres des fonctions pour générer les requêtes : 
#a bien définir !!
nb_users = 700
max_requetes = 4 
max_jours = 44
threshold_city = 0.2
threshold_language = 0.1

## Fonction de génération des requêtes 

In [212]:
# Fonction pour générer toutes les requêtes d'un utilisateur : 

def all_requests_1user(user_ID,max_requetes,max_jours,threshold_city,threshold_language) : 
    # initialisation dataframe pour stocker la liste des requêtes + définir le nombre de requêtes faites : 
    df_user = pd.DataFrame()
    nb_requetes = random.randint(1,max_requetes)

    # Enregistrement du user_ID sur toutes les lignes du df
    df_user["avatar_name"] = [user_ID]*nb_requetes
    
    # Attribution d'une date 
    #--Choix des bornes 
    dates = np.sort(random.choices(list(range(max_jours)), k = nb_requetes))[::-1]
    df_user["date"] = dates
        
    # Attribution de la ville 
    city_user = random.choice(city) 
    prob = np.random.uniform(0,1,nb_requetes)
    vect_city = pd.Series([""]*nb_requetes)
    
    vect_city[prob >= threshold_city] = city_user
    vect_city[prob < threshold_city] = random.choice(city) 
    df_user["city"] = vect_city
    
    # Attribution de la langue : 
    # 1. une langue pour l'utilisateur :
    language_user = random.choice(language) 
    vect_language = pd.Series([language_user]*nb_requetes)
    
    # 2. remplacer cette langue par une langue similaire ? 
    if language_user in dic_similar_language : 
        prob = np.random.uniform(0,1,nb_requetes)
        ind = np.where(prob<=threshold_language)[0]
        vect_language[ind] = random.choice(dic_similar_language[language_user])
    
    # 3. remplacer cette langue par la langue du pays dans lequel l'utilisateur se rend ? 
    prob = np.random.uniform(0,1,nb_requetes)
    ind = np.where(prob<=threshold_language)[0]
    vect_language[ind] = dic_language_country[vect_city[ind]]
    df_user["language"] = vect_language
    
    # Attribution du mobile 
    df_user["mobile"] = random.choices([0,1],k=nb_requetes)
    return df_user

df1 = all_requests_1user(1,max_requetes,max_jours,threshold_city,threshold_language)

In [213]:
# Fonction pour créer un df pour stocker toutes les requêtes de tous les  utilisateurs :
def all_requests_all_users(nb_users,max_requetes,max_jours,threshold_city,threshold_language) : 
    df = pd.DataFrame(columns=np.r_[["avatar_name"],request_var])
    for i in range(nb_users) :
        # génération d'un df par user 
        df_1user = all_requests_1user(i,max_requetes,max_jours,threshold_city,threshold_language)
        # concaténation de tous les dataframes en un seul
        df = pd.concat([df,df_1user],ignore_index=True)
    return df 

df = all_requests_all_users(nb_users,max_requetes,max_jours,threshold_city,threshold_language) 